In [1]:
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from env import user, password, host
from scipy import stats
from scipy.stats import levene, ttest_ind
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
import math
import numpy as np
import os
import pandas as pd
import requests
import seaborn as sns
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
import wrangle as w


# Acquire

### Data was acquired from the Texas Education Agency website

In [2]:
df22 = pd.read_csv('CAMPUS_summary_22.csv')
df21 = pd.read_csv('CAMPUS_summary_21.csv')
df20 = pd.read_csv('CAMPUS_summary_20.csv')
df19 = pd.read_csv('CAMPUS_summary_19.csv')
df18 = pd.read_csv('CAMPUS_summary_18.csv')

In [3]:
df22.head()

,agg_level,campus_number,region,dist_name_num,charter_status,campus,section,heading,heading_name,student_count
0,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,533
1,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A02,CAMPUS DISCIPLINE POPULATION,27
2,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,40
3,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,B-DISCIPLINE DATA TRENDS,B13,STUDENTS SUSPENDED OUT OF SCHOOL,27
4,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,F-OUT OF SCHOOL SUSPENSIONS,C24,BLACK OR AFRICAN AMERICAN,-999


In [4]:
df21.head()

,agg_level,campus_number,region,dist_name_num,charter_status,campus,section,heading,heading_name,student_count
0,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,853
1,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,QUEST CAMPUS PRE-K THRU 8 057816101,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,682
2,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,QUEST CAMPUS PRE-K THRU 8 057816101,A-PARTICIPATION,A02,CAMPUS DISCIPLINE POPULATION,-999
3,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,QUEST CAMPUS PRE-K THRU 8 057816101,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,-999
4,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,QUEST CAMPUS PRE-K THRU 8 057816101,B-DISCIPLINE DATA TRENDS,B13,STUDENTS SUSPENDED OUT OF SCHOOL,-999


In [5]:
df20.head()

,agg_level,campus_number,region,dist_name_num,charter_status,campus,section,heading,heading_name,student_count
0,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,1086
1,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A02,CAMPUS DISCIPLINE POPULATION,20
2,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,32
3,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,B-DISCIPLINE DATA TRENDS,B10,COUNT OF STUDENTS SUSPENDED IN SCHOOL,-999
4,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,B-DISCIPLINE DATA TRENDS,B13,STUDENTS SUSPENDED OUT OF SCHOOL,16


In [6]:
df19.head()

,agg_level,campus_number,region,dist_name_num,charter_status,campus,section,heading,heading_name,student_count
0,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,A W BROWN - F L A EARLY CHILDHOOD 057816102,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,1291
1,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,AW BROWN-F L A INT CAMPUS 057816101,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,961
2,CAMPUS SUMMARY,57829001,10,A+ ACADEMY 057829,OPEN ENROLLMENT CHARTER,A+ ACADEMY EL 057829001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,853
3,CAMPUS SUMMARY,57829001,10,A+ ACADEMY 057829,OPEN ENROLLMENT CHARTER,A+ ACADEMY EL 057829001,A-PARTICIPATION,A02,CAMPUS DISCIPLINE POPULATION,24
4,CAMPUS SUMMARY,57829001,10,A+ ACADEMY 057829,OPEN ENROLLMENT CHARTER,A+ ACADEMY EL 057829001,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,26


In [7]:
df18.head()

,agg_level,campus_number,region,dist_name_num,charter_status,campus,section,heading,heading_name,student_count
0,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,A W BROWN - F L A EARLY CHILDHOOD 057816102,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,1278
1,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,AW BROWN-F L A INT CAMPUS 057816101,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,1106
2,CAMPUS SUMMARY,57829001,10,A+ ACADEMY 057829,OPEN ENROLLMENT CHARTER,A+ ACADEMY EL 057829001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,859
3,CAMPUS SUMMARY,57829002,10,A+ ACADEMY 057829,OPEN ENROLLMENT CHARTER,A+ SECONDARY SCHOOL 057829002,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,584
4,CAMPUS SUMMARY,57829001,10,A+ ACADEMY 057829,OPEN ENROLLMENT CHARTER,A+ ACADEMY EL 057829001,A-PARTICIPATION,A02,CAMPUS DISCIPLINE POPULATION,-999


In [8]:
df22

,agg_level,campus_number,region,dist_name_num,charter_status,campus,section,heading,heading_name,student_count
0,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,533
1,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A02,CAMPUS DISCIPLINE POPULATION,27
2,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,40
3,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,B-DISCIPLINE DATA TRENDS,B13,STUDENTS SUSPENDED OUT OF SCHOOL,27
4,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,F-OUT OF SCHOOL SUSPENSIONS,C24,BLACK OR AFRICAN AMERICAN,-999
...,...,...,...,...,...,...,...,...,...,...
323988,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,V-AT RISK IN SCHOOL SUS.,F19,NON AT RISK IN SCHOOL SUSPENSIONS,-999
323989,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,W-REASON INCIDENT COUNTS,G21,21-VIOLATED LOCAL CODE OF CONDUCT,-999
323990,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,W-REASON INCIDENT COUNTS,G61,61-BULLYING  TEC 37.0052(B),-999
323991,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,X-DISCIPLINE ACTION COUNTS,H05,05-OUT-OF-SCHOOL SUSPENSION,-999


In [9]:
w.prep22(df22)

NameError: name 'df22' is not defined

In [ ]:
df22.head()

In [ ]:
stop

In [ ]:
def prep21(df):
    global df21
    df21['charter_encoded'] = df21.charter_status.map({'OPEN ENROLLMENT CHARTER': 1, 'TRADITIONAL ISD/CSD':0})
    df21=df21[(df21.heading == 'A01') | (df21.heading ==  'A03')]
    df21=df21[df21['student_count'] != '-999']
    df21['student_count']= df21['student_count'].str.replace("<", "")
    df21['student_count'] = df21['student_count'].astype(float)
    dfpivot=df21.pivot(index='campus_number', columns='heading', values= 'student_count').dropna()
    df21=df21.merge(dfpivot,how= 'right', on= 'campus_number')
    df21=df21.rename(columns={'A01': 'student_enrollment', 'A03':'discipline_count'})
    df21['discipline_percent']= ((df21['discipline_count']/df21['student_enrollment'])*100)
    df21=df21.round({'discipline_percent': 0})
    df21=df21.drop(columns=['agg_level', 'campus_number', 'region', 'charter_status', 'dist_name_num', 'student_count','section', 'heading', 'heading_name', 'student_count'])
    df21=df21.drop_duplicates()
    df21=df21.reset_index(drop=True)

In [ ]:
prep21(df21)

In [ ]:
df21.head()

In [ ]:
def prep20(df):
    global df20
    df20['charter_encoded'] = df20.charter_status.map({'OPEN ENROLLMENT CHARTER': 1, 'TRADITIONAL ISD/CSD':0})
    df20=df20[(df20.heading == 'A01') | (df20.heading ==  'A03')]
    df20=df20[df20['student_count'] != '-999']
    df20['student_count']= df20['student_count'].str.replace("<", "")
    df20['student_count'] = df20['student_count'].astype(float)
    dfpivot=df20.pivot(index='campus_number', columns='heading', values= 'student_count').dropna()
    df20=df20.merge(dfpivot,how= 'right', on= 'campus_number')
    df20=df20.rename(columns={'A01': 'student_enrollment', 'A03':'discipline_count'})
    df20['discipline_percent']= ((df20['discipline_count']/df20['student_enrollment'])*100)
    df20=df20.round({'discipline_percent': 0})
    df20=df20.drop(columns=['agg_level', 'campus_number', 'region', 'charter_status', 'dist_name_num', 'student_count','section', 'heading', 'heading_name', 'student_count'])
    df20=df20.drop_duplicates()
    df20=df20.reset_index(drop=True)

In [ ]:
prep20(df20)

In [ ]:
df20.head()

In [ ]:
def prep19(df):
    global df19
    df19['charter_encoded'] = df19.charter_status.map({'OPEN ENROLLMENT CHARTER': 1, 'TRADITIONAL ISD/CSD':0})
    df19=df19[(df19.heading == 'A01') | (df19.heading ==  'A03')]
    df19=df19[df19['student_count'] != '-999']
    df19['student_count']= df19['student_count'].str.replace("<", "")
    df19['student_count'] = df19['student_count'].astype(float)
    dfpivot=df19.pivot(index='campus_number', columns='heading', values= 'student_count').dropna()
    df19=df19.merge(dfpivot,how= 'right', on= 'campus_number')
    df19=df19.rename(columns={'A01': 'student_enrollment', 'A03':'discipline_count'})
    df19['discipline_percent']= ((df19['discipline_count']/df19['student_enrollment'])*100)
    df19=df19.round({'discipline_percent': 0})
    df19=df19.drop(columns=['agg_level', 'campus_number', 'region', 'charter_status', 'dist_name_num', 'student_count','section', 'heading', 'heading_name', 'student_count'])
    df19=df19.drop_duplicates()
    df19=df19.reset_index(drop=True)

In [ ]:
prep19(df19)

In [ ]:
df19.head()

In [ ]:
def prep18(df):
    global df18
    df18['charter_encoded'] = df18.charter_status.map({'OPEN ENROLLMENT CHARTER': 1, 'TRADITIONAL ISD/CSD':0})
    df18=df18[(df18.heading == 'A01') | (df18.heading ==  'A03')]
    df18=df18[df18['student_count'] != '-999']
    df18['student_count']= df18['student_count'].str.replace("<", "")
    df18['student_count'] = df18['student_count'].astype(float)
    dfpivot=df18.pivot(index='campus_number', columns='heading', values= 'student_count').dropna()
    df18=df18.merge(dfpivot,how= 'right', on= 'campus_number')
    df18=df18.rename(columns={'A01': 'student_enrollment', 'A03':'discipline_count'})
    df18['discipline_percent']= ((df18['discipline_count']/df18['student_enrollment'])*100)
    df18=df18.round({'discipline_percent': 0})
    df18=df18.drop(columns=['agg_level', 'campus_number', 'region', 'charter_status', 'dist_name_num', 'student_count','section', 'heading', 'heading_name', 'student_count'])
    df18=df18.drop_duplicates()
    df18=df18.reset_index(drop=True)

In [ ]:
prep18(df18)

In [ ]:
df18.head()

In [ ]:
def df_combine(a,b,c,d,e):
    df=pd.concat([df18,df19,df20,df21,df22], ignore_index=True)
    return(df)

In [ ]:
df=df_combine(df18,df19,df20,df21,df22)

In [ ]:
df.head()

In [ ]:
def split_tea_data(df):
    '''
    This function performs split on tea data, stratify charter_encoded.
    Returns train, validate, and test dfs.
    '''
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123, 
                                        stratify=df.charter_encoded)
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123, 
                                   stratify=train_validate.charter_encoded)
    return train, validate, test

In [ ]:
train, validate, test=split_tea_data(df)

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
validate.shape

In [ ]:
test.shape

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.columns

In [ ]:
sns.pairplot(data=train, kind="reg", corner=True)

In [ ]:
num_vars = ['student_enrollment', 'discipline_count' 'discipline_percent']
cat_vars = ['campus', 'charter_encoded']

In [ ]:
for col in train.columns:
    plt.hist(train[col])
    plt.title(col)
    plt.show()

In [ ]:
for col in cat_vars:
    print(col)
    print(train[col].value_counts())
    print(train[col].value_counts(normalize=True)*100)
    sns.countplot(x=col, data=train)
    plt.show()

In [ ]:
978/15918

In [ ]:
14940/15918

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10))
sns.boxplot(x=train['charter_encoded'], y=train['discipline_percent']);
plt.xticks(rotation = 90)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10))
sns.boxplot(x=train['charter_encoded'], y=train['discipline_count']);
plt.xticks(rotation = 90)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10))
sns.boxplot(x=train['charter_encoded'], y=train['student_enrollment']);
plt.xticks(rotation = 90)
plt.show()

In [ ]:
sns.scatterplot(x='student_enrollment', y='discipline_count', data=train, hue='charter_encoded')

In [ ]:
sns.histplot(x='discipline_percent', data=train, kde=True)

In [ ]:
# histogram of churned customers
charterdiscp = train[train.charter_encoded==1].discipline_percent
charterdiscp.hist()

In [ ]:
# histogram of non-churned customers
non_charterdiscp = train[train.charter_encoded==0].discipline_percent
non_charterdiscp.hist()

Hypothesis
Null H: Mean of disciplined students is the same for charter schools and traditional schools. Alt H: Mean of disciplined students is not the same for charter schools and traditional schools.

In [ ]:
mean_a = charterdiscp.mean()
mean_b = non_charterdiscp.mean()

In [ ]:
mean_a, mean_b

In [ ]:
# remember that our confidence level determines our alpha, 
# which we use to compare to p
alpha = 0.05

In [ ]:
# Mini H0: variances are equal between groups
# Ha: variances are inequal between groups
# alpha: 0.05
stat, pval = stats.levene(charterdiscp, non_charterdiscp)

In [ ]:
stat, pval

In [ ]:
# implications of one-tailed two-sampled t-test:
# p will be divided by two because we are only looking at one tail of the curve
# we will pay attention to the sign of our t-statistic
# t, p = t-test: (group A, group B) ==> comparison of mu_a > mu_b
t, p = stats.ttest_ind(charterdiscp, non_charterdiscp, equal_var=False)

In [ ]:
t, p

In [ ]:
print('Is p less than alpha? ', pval < alpha)

In [ ]:
if (t > 0) and ((p / 2) < alpha):
    print('We can reject the null hypothesis')
else:
    print('we fail to reject the null hypothesis')

In [ ]:
# assign the train variables
X_train_scaled = train[['student_enrollment','discipline_count','discipline_percent']]
y_train=train[['charter_encoded']]

#assign the validate variables
X_validate_scaled=validate[['student_enrollment','discipline_count','discipline_percent']]
y_validate=validate[['charter_encoded']]

# assign the test variables
X_test_scaled=test[['student_enrollment','discipline_count','discipline_percent']]
y_test=test[['charter_encoded']]

In [ ]:
def scale_data(train, 
               validate, 
               test, 
               columns_to_scale = ['student_enrollment','discipline_count','discipline_percent'],
               return_scaler=False):
    '''
    Scales the 3 data splits. 
    Takes in train, validate, and test data splits and returns their scaled counterparts.
    If return_scalar is True, the scaler object will be returned as well
    '''
    # make copies of our original data so we dont gronk up anything
    train_scaled = train.copy()
    validate_scaled = validate.copy()
    test_scaled = test.copy()
    #     make the thing
    scaler = MinMaxScaler()
    #     fit the thing
    scaler.fit(train[columns_to_scale])
    # applying the scaler:
    train_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(train[columns_to_scale]),
                                                  columns=train[columns_to_scale].columns.values).set_index([train.index.values])
                                                  
    validate_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(validate[columns_to_scale]),
                                           columns=validate[columns_to_scale].columns.values).set_index([validate.index.values])
    
    test_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(test[columns_to_scale]),
                                                 columns=test[columns_to_scale].columns.values).set_index([test.index.values])
    
    if return_scaler:
        return scaler, train_scaled, validate_scaled, test_scaled
    else:
        return train_scaled, validate_scaled, test_scaled

In [ ]:
train_scaled, validate_scaled, test_scaled = scale_data(train, validate, test)

In [ ]:
train_scaled, validate_scaled, test_scaled

In [ ]:
X_train_scaled.head()

In [ ]:
y_train.value_counts()

## Baseline

In [ ]:
# The mode is a great baseline
baseline = y_train.mode()

In [ ]:
baseline

In [ ]:
X_train_scaled['baseline_prediction'] = 0
X_train_scaled

In [ ]:
actual = y_train
predictions = X_train_scaled.baseline_prediction

In [ ]:
pd.crosstab(y_train['charter_encoded'], X_train_scaled.baseline_prediction)

In [ ]:
# Creates a boolean array where "No" becomes True and "Yes" becomes False
(y_train == 0)

In [ ]:
# Takes the .mean() of the boolean array that represents the proportion 
# that would also match our baseline prediction
(y_train == 0).mean() 

In [ ]:
X_validate_scaled['baseline_prediction'] = 0

In [ ]:
X_test_scaled['baseline_prediction'] = 0

# Model: Decision Tree

## decision tree max depth=3

In [ ]:
# Create the decision tree object and specify hyperparams
tree3 = DecisionTreeClassifier(max_depth = 3, random_state=123)

In [ ]:
tree3.fit(X_train_scaled, y_train)

# Rudimentary visualization of model structure
print(export_text(tree3, feature_names=X_train_scaled.columns.tolist()))

In [ ]:
# Visualize the tree
plt.figure(figsize=(12, 7))
plot_tree(tree3, feature_names=X_train_scaled.columns, class_names=tree3.classes_.astype(str))
#plt.show()

In [ ]:
y_pred3=tree3.predict(X_train_scaled)

In [ ]:
y_pred_proba3 = tree3.predict_proba(X_train_scaled)


In [ ]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(tree3.score(X_train_scaled, y_train)))

# Create a confusion matrix

In [ ]:
tree3.predict(X_train_scaled)

In [ ]:
predictions3 = tree3.predict(X_train_scaled)
actual3 = y_train

In [ ]:
confusion_matrix(actual, predictions)

In [ ]:
pd.crosstab(y_train['charter_encoded'], tree3.predict(X_train_scaled))

In [ ]:
plot_confusion_matrix(tree3, X_train_scaled, y_train, display_labels=['traditional', 'charter'])


In [ ]:
print(classification_report(actual3, predictions3))

## decision tree max depth=4

In [ ]:
# Create the decision tree object and specify hyperparams
tree4 = DecisionTreeClassifier(max_depth = 4, random_state=123)

In [ ]:
tree4

In [ ]:
tree4.fit(X_train_scaled, y_train)

In [ ]:
# Rudimentary visualization of model structure
print(export_text(tree4, feature_names=X_train_scaled.columns.tolist()))

In [ ]:
# Visualize the tree
plt.figure(figsize=(12, 7))
plot_tree(tree4, feature_names=X_train_scaled.columns, class_names=tree4.classes_.astype(str))
#plt.show()

In [ ]:
y_pred4=tree4.predict(X_train_scaled)

In [ ]:
y_pred_proba4 = tree4.predict_proba(X_train_scaled)

In [ ]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(tree4.score(X_train_scaled, y_train)))

In [ ]:
tree4.predict(X_train_scaled)

In [ ]:
predictions4 = tree4.predict(X_train_scaled)

In [ ]:
actual4 = y_train

In [ ]:
confusion_matrix(actual4, predictions4)

In [ ]:
pd.crosstab(y_train['charter_encoded'], tree4.predict(X_train_scaled))

In [ ]:
plot_confusion_matrix(tree4, X_train_scaled, y_train, display_labels=['traditional', 'charter'])


In [ ]:
print(classification_report(actual4, predictions4))

## decision tree max depth=5

In [ ]:
# Create the decision tree object and specify hyperparams
tree5 = DecisionTreeClassifier(max_depth = 5, random_state=123)

In [ ]:
tree5.fit(X_train_scaled, y_train)

# Rudimentary visualization of model structure
print(export_text(tree5, feature_names=X_train_scaled.columns.tolist()))

In [ ]:
# Visualize the tree
plt.figure(figsize=(12, 7))
plot_tree(tree5, feature_names=X_train_scaled.columns, class_names=tree5.classes_.astype(str))
#plt.show()

In [ ]:
y_pred5=tree5.predict(X_train_scaled)

In [ ]:
y_pred_proba5 = tree5.predict_proba(X_train_scaled)

In [ ]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(tree5.score(X_train_scaled, y_train)))

In [ ]:
tree5.predict(X_train_scaled)

In [ ]:
predictions5 = tree5.predict(X_train_scaled)
actual5 = y_train

In [ ]:
confusion_matrix(actual5, predictions5)


In [ ]:
pd.crosstab(y_train['charter_encoded'], tree5.predict(X_train_scaled))


In [ ]:
plot_confusion_matrix(tree5, X_train_scaled, y_train, display_labels=['traditional', 'charter'])


In [ ]:
print(classification_report(actual5, predictions5))


## best decision tree depth =4

In [ ]:
print(classification_report(actual4, predictions4))

# Random Forest

## RF max depth=3

In [ ]:
rf3 = RandomForestClassifier(max_depth=3, random_state=123)
rf3

In [ ]:
rf3.fit(X_train_scaled, y_train)

In [ ]:
print(rf3.feature_importances_)

In [ ]:
plt.bar(X_train_scaled.columns, rf3.feature_importances_)
plt.show()

In [ ]:
y_pred3 = rf3.predict(X_train_scaled)
y_pred3

In [ ]:
y_pred_proba3 = rf3.predict_proba(X_train_scaled)
y_pred_proba3

In [ ]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf3.score(X_train_scaled, y_train)))

In [ ]:
cm3 = confusion_matrix(y_train, y_pred3)
print(cm3)

In [ ]:
disp3 = ConfusionMatrixDisplay(confusion_matrix=cm3, display_labels=rf3.classes_)
disp3.plot()
plt.show()

In [ ]:
print(classification_report(y_train, y_pred3))

## RF max depth=4

In [ ]:
rf4 = RandomForestClassifier(max_depth=4, random_state=123)
rf4

In [ ]:
rf4.fit(X_train_scaled, y_train)

In [ ]:
print(rf4.feature_importances_)

In [ ]:
plt.bar(X_train_scaled.columns, rf4.feature_importances_)
plt.show()

In [ ]:
y_pred4 = rf4.predict(X_train_scaled)
y_pred4

In [ ]:
y_pred_proba4 = rf4.predict_proba(X_train_scaled)
y_pred_proba4

In [ ]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf4.score(X_train_scaled, y_train)))

In [ ]:
cm4 = confusion_matrix(y_train, y_pred4)
print(cm4)

In [ ]:
disp4 = ConfusionMatrixDisplay(confusion_matrix=cm4, display_labels=rf4.classes_)
disp4.plot()
plt.show()

In [ ]:
print(classification_report(y_train, y_pred4))

## RF max depth=5

In [ ]:
rf5 = RandomForestClassifier(max_depth=5, random_state=123)
rf5

In [ ]:
rf5.fit(X_train_scaled, y_train)

In [ ]:
print(rf5.feature_importances_)

In [ ]:
plt.bar(X_train_scaled.columns, rf5.feature_importances_)
plt.show()

In [ ]:
y_pred5 = rf5.predict(X_train_scaled)
y_pred5

In [ ]:
y_pred_proba5 = rf5.predict_proba(X_train_scaled)
y_pred_proba5

In [ ]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf5.score(X_train_scaled, y_train)))

In [ ]:
cm5 = confusion_matrix(y_train, y_pred5)
print(cm5)

In [ ]:
disp5 = ConfusionMatrixDisplay(confusion_matrix=cm5, display_labels=rf5.classes_)
disp5.plot()
plt.show()

In [ ]:
print(classification_report(y_train, y_pred5))

# K-Nearest Neighbor

## n_neighbors=3

In [ ]:
# weights = ['uniform', 'density']
knn3 = KNeighborsClassifier(n_neighbors=3, weights='uniform')

In [ ]:
knn3.fit(X_train_scaled, y_train)

In [ ]:
y_pred3 = knn3.predict(X_train_scaled)

In [ ]:
y_pred_proba3 = knn3.predict_proba(X_train_scaled)

In [ ]:
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn3.score(X_train_scaled, y_train)))

In [ ]:
print(confusion_matrix(y_train, y_pred3))

In [ ]:
print(classification_report(y_train, y_pred3))

## n_neighbors=4

In [ ]:
# weights = ['uniform', 'density']
knn4 = KNeighborsClassifier(n_neighbors=4, weights='uniform')

In [ ]:
knn4.fit(X_train_scaled, y_train)

In [ ]:
y_pred4 = knn4.predict(X_train_scaled)

In [ ]:
y_pred_proba4 = knn4.predict_proba(X_train_scaled)

In [ ]:
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn4.score(X_train_scaled, y_train)))

In [ ]:
print(confusion_matrix(y_train, y_pred4))

In [ ]:
print(classification_report(y_train, y_pred4))

## n_neighbors=5

In [ ]:
# weights = ['uniform', 'density']
knn5 = KNeighborsClassifier(n_neighbors=5, weights='uniform')

In [ ]:
knn5.fit(X_train_scaled, y_train)

In [ ]:
y_pred5 = knn5.predict(X_train_scaled)

In [ ]:
y_pred_proba5 = knn5.predict_proba(X_train_scaled)

In [ ]:
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn5.score(X_train_scaled, y_train)))

In [ ]:
print(confusion_matrix(y_train, y_pred5))

In [ ]:
print(classification_report(y_train, y_pred5))

# Logistic Regression

In [ ]:
# Define the logistic regression model
logit = LogisticRegression(C=1, random_state=123)

In [ ]:
#  fit the model on train data
logit.fit(X_train_scaled, y_train)

In [ ]:
# now use the model to make predictions
y_predlr = logit.predict(X_train_scaled)

In [ ]:
#take a look at predictions
y_predlr

In [ ]:
# look at predicted probabilites for first 10 observations
logit.predict_proba(X_train_scaled)[:10]

In [ ]:
logit.classes_

In [ ]:
# View raw probabilities (output from the model)

y_pred_probalr = logit.predict_proba(X_train_scaled)
y_pred_probalr = pd.DataFrame(y_pred_proba, columns = ['traditional', 'charter'])
y_pred_probalr.head().round(3)

In [ ]:
#classification report
print(classification_report(y_train, y_predlr))

In [ ]:
# look at model 1 coefficents
 
print('Coefficient: \n', logit.coef_[0])

In [ ]:
# Make a dataframe of coefficients and feature names

log_coeffs = pd.DataFrame(logit.coef_[0], index = X_train_scaled.columns,
                          columns = ['coeffs']).sort_values(by = 'coeffs', ascending = True)
log_coeffs

In [ ]:
# convert from log odds to odds (exponentiate)
odds = np.exp(log_coeffs)
odds


# classification report

There are four ways to check if the predictions are right or wrong:

TN / True Negative: the case was negative and predicted negative
TP / True Positive: the case was positive and predicted positive
FN / False Negative: the case was positive but predicted negative
FP / False Positive: the case was negative but predicted positive



Precision — What percent of your predictions were correct?
Precision:- Accuracy of positive predictions.

Precision = TP/(TP + FP)


Recall — What proportion of actual positives was identified correctly?
Recall:- Fraction of positives that were correctly identified.

Recall = TP/(TP+FN)

F1 score — What percent of positive predictions were correct?
F1 Score is the weighted average of Precision and Recall. Therefore, this score takes both false positives and false negatives into account. Intuitively it is not as easy to understand as accuracy, but F1 is usually more useful than accuracy, especially if you have an uneven class distribution

F1 Score = 2*(Recall * Precision) / (Recall + Precision)


Support — Support is the number of actual occurrences of the class in the specified dataset.



# Validate

In [ ]:
# Create the decision tree object and specify hyperparams
tree4 = DecisionTreeClassifier(max_depth = 4, random_state=123)

In [ ]:
tree4.fit(X_validate_scaled, y_validate)

In [ ]:
# Rudimentary visualization of model structure
print(export_text(tree4, feature_names=X_validate_scaled.columns.tolist()))

In [ ]:
#Visualize the tree
plt.figure(figsize=(12, 7))
plot_tree(tree4, feature_names=X_validate_scaled.columns, class_names=tree4.classes_.astype(str))
plt.show()

In [ ]:
y_pred4=tree4.predict(X_validate_scaled)

In [ ]:
y_pred_proba4 = tree4.predict_proba(X_validate_scaled)

In [ ]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(tree4.score(X_validate_scaled, y_validate)))

In [ ]:
tree4.predict(X_validate_scaled)

In [ ]:
predictions4 = tree4.predict(X_validate_scaled)

In [ ]:
actual4 = y_validate

In [ ]:
confusion_matrix(actual4, predictions4)

In [ ]:
pd.crosstab(y_validate['charter_encoded'], tree4.predict(X_validate_scaled))

In [ ]:
plot_confusion_matrix(tree4, X_validate_scaled, y_validate, display_labels=['traditional', 'charter'])

In [ ]:
print(classification_report(actual4, predictions4))

# Test

In [ ]:
# Create the decision tree object and specify hyperparams
tree4 = DecisionTreeClassifier(max_depth = 4, random_state=123)

In [ ]:
tree4.fit(X_test_scaled, y_test)

In [ ]:
# Rudimentary visualization of model structure
print(export_text(tree4, feature_names=X_train_scaled.columns.tolist()))

In [ ]:
#Visualize the tree
plt.figure(figsize=(12, 7))
plot_tree(tree4, feature_names=X_test_scaled.columns, class_names=tree4.classes_.astype(str))
plt.show()

In [ ]:
y_pred4=tree4.predict(X_test_scaled)

In [ ]:
y_pred_proba4 = tree4.predict_proba(X_test_scaled)

In [ ]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(tree4.score(X_test_scaled, y_test)))

In [ ]:
tree4.predict(X_test_scaled)

In [ ]:
predictions4 = tree4.predict(X_test_scaled)

In [ ]:
actual4 = y_test

In [ ]:
confusion_matrix(actual4, predictions4)

pd.crosstab(y_test['charter_encoded'], tree4.predict(X_test_scaled))

In [ ]:
plot_confusion_matrix(tree4, X_test_scaled, y_test, display_labels=['traditional', 'charter'])

In [ ]:
print(classification_report(actual4, predictions4))

In [ ]:
pred_dict = {
    'model': [],
    'accuracy': [],
    'true_positive_rate': [],
    'false_positive_rate': [],
    'true_negative_rate': [],
    'false_negative_rate': [],
    'precision': [],
    'recall': [],
    'f1_score': [],
    'support_0': [],
    'support_1': []
}

for n in [4]:
    tree4 = DecisionTreeClassifier(max_depth = 4, random_state=123)
    
    tree4.fit(X_test_scaled, y_test)
    
    y_pred4=tree4.predict(X_test_scaled)
    
    TN, FP, FN, TP = confusion_matrix(y_test, y_pred4).ravel()
    ALL = TP + TN + FP + FN
    
    accuracy = (TP + TN)/ALL
    true_positive_rate = TP/(TP+FN)
    false_positive_rate = FP/(FP+TN)
    true_negative_rate = TN/(TN+FP)
    false_negative_rate = FN/(FN+TP)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    f1_score = 2*(precision*recall)/(precision+recall)
    support_pos = TP + FN
    support_neg = FP + TN
    
    pred_dict['model'].append(f'decision_tree_{n}')
    pred_dict['accuracy'].append(accuracy)
    pred_dict['true_positive_rate'].append(true_positive_rate)
    pred_dict['false_positive_rate'].append(false_positive_rate)
    pred_dict['true_negative_rate'].append(true_negative_rate)
    pred_dict['false_negative_rate'].append(false_negative_rate)
    pred_dict['precision'].append(precision)
    pred_dict['recall'].append(recall)
    pred_dict['f1_score'].append(f1_score)
    pred_dict['support_0'].append(support_pos)
    pred_dict['support_1'].append(support_neg)

In [ ]:
pd.DataFrame(pred_dict)

In [ ]:
pd.DataFrame(pred_dict).T